In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import cv2
import os
import mediapipe as mp
import pickle

from PIL import ImageOps, Image
from tqdm import tqdm
from collections import deque, Counter
from tensorflow.keras.models import load_model

2022-06-19 17:24:06.825427: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


## Metrics

from: https://www.mdpi.com/2414-4088/5/9/55
section 4.3.2 repetition counting  
*OBO is
above 99%, denoting that almost all the test samples are within +-1 of groundtruth.*

In [2]:
def OBO(y_pred: int, y_true: int) -> int:
    return int(y_pred >= y_true-1 and y_pred <= y_true+1)

In [3]:
OBO(9,10)

1

## Custom Objects for ViViT

In [4]:
from tensorflow.keras import layers

In [5]:
class PositionalEncoder(layers.Layer):
    def __init__(self, embed_dim, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        
    def build(self, input_shape):
        _, num_tokens, _ = input_shape
        self.position_embedding = layers.Embedding(
            input_dim=num_tokens, output_dim=self.embed_dim
        )
        self.positions = tf.range(start=0, limit=num_tokens, delta=1)
        
    def call(self, encoded_tokens):
        # Encode the positions and add it to the encoded tokens
        encoded_positions = self.position_embedding(self.positions)
        encoded_tokens = encoded_tokens + encoded_positions
        return encoded_tokens
    
    def get_config(self):
        config = super().get_config()
        config.update({
            'embed_dim': self.embed_dim,
        })
        return config

In [6]:
class TubeletEmbedding(layers.Layer):
    def __init__(self, embed_dim, patch_size, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.patch_size = patch_size
        self.projection = layers.Conv3D(
            filters=embed_dim,
            kernel_size=patch_size,
            strides=patch_size,
            padding='VALID'
        )
        self.flatten = layers.Reshape(target_shape=(-1, embed_dim))
        
    def call(self, videos):
        projected_patches = self.projection(videos)
        flattened_patches = self.flatten(projected_patches)
        return flattened_patches
    
    def get_config(self):
        config = super().get_config()
        config.update({
            'embed_dim': self.embed_dim,
            'patch_size': self.patch_size,
        })
        return config

# Classification

In [7]:
mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic
mp_pose = mp.solutions.pose

In [8]:
# config
ROUND_MAPPING = {
    'pushup': 8,
    'squat': 6,
    'jumping-jack': 4,
    'leg-raise': 3,
    'half-burpee': 3,
}
EXERCISE = 'squat'
DS_TYPE = 'images'
SEQ_LEN = 'single'
# test_walk_dir = f'/data1-6tb/jom/Dataset/EzFit_dataset/test/{DS_TYPE}/len_{SEQ_LEN}/{EXERCISE}'
label_file = f'/data1-6tb/jom/Dataset/EzFit_dataset/labels/test/{DS_TYPE}/len_{SEQ_LEN}/{EXERCISE}_full_cls_label.csv'
output_dir = f'/data1-6tb/jom/Dataset/EzFit_dataset/outputs/test/{DS_TYPE}/len_{SEQ_LEN}'
ds_base_dir = f'/data1-6tb/jom/Dataset/EzFit_dataset/test'

In [9]:
# model related
BENCHMARK = True
MODEL_TYPE = 'kps_single'

r = ROUND_MAPPING[EXERCISE]
if BENCHMARK:
    r = 'BM_BARE'
    CUR_CLASS = 'up'
    PREV_CLASS = 'down'
    BASE_CLASSES = [f'{EXERCISE}-{CUR_CLASS}',f'{EXERCISE}-{PREV_CLASS}']

model_path = f'/data1-6tb/jom/EzFit_models/saved_models/len_{SEQ_LEN}/round_{r}/{EXERCISE}/{EXERCISE}_{MODEL_TYPE}.h5'
if MODEL_TYPE == 'kps_single':
    le_path = f'/data1-6tb/jom/EzFit_models/saved_pickles/len_{SEQ_LEN}/round_{r}/{EXERCISE}/{EXERCISE}_{MODEL_TYPE}_lb.pickle'
else:
    le_path = f'/data1-6tb/jom/EzFit_models/saved_pickles/len_{SEQ_LEN}/round_{r}/{EXERCISE}/{EXERCISE}_{MODEL_TYPE}_le.pickle'

if MODEL_TYPE == 'ViViT':
    custom_objects = {"TubeletEmbedding": TubeletEmbedding, 
                      "PositionalEncoder": PositionalEncoder}
else:
    custom_objects = {}

In [10]:
# load model and le
loaded_model = load_model(model_path, custom_objects=custom_objects)
with open(le_path, 'rb') as file:
    loaded_le = pickle.load(file)

2022-06-19 17:24:18.922515: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2022-06-19 17:24:19.021918: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-19 17:24:19.022459: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:04:00.0 name: GeForce RTX 2080 SUPER computeCapability: 7.5
coreClock: 1.845GHz coreCount: 48 deviceMemorySize: 7.79GiB deviceMemoryBandwidth: 462.00GiB/s
2022-06-19 17:24:19.022502: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-19 17:24:19.023616: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 1 with properties: 
pciBusID: 000

In [11]:
if MODEL_TYPE == 'kps_single':
    pass
else:
    label_df = pd.read_csv(label_file)
    label_df

In [12]:
test_arr = []
error_files = [] # only for kps_single
labels = [] # only for kps_single
if MODEL_TYPE == 'kps_single':
    accepted_exts = ('.jpg', '.jpeg','.png')
    for path, subdirs, files in tqdm(os.walk(os.path.join(ds_base_dir, 'images', EXERCISE))):
        for name in files:
            if name.lower().endswith(accepted_exts):
                full_path = os.path.join(path, name)
                split = full_path.split(os.path.sep)
                if BENCHMARK:
                    if split[-2] in BASE_CLASSES:
                        image = Image.open(full_path)
                        image = np.array(image)
                        label = split[-2]
                        with mp_pose.Pose(min_detection_confidence = 0.5, min_tracking_confidence = 0.5) as pose:

                            results = pose.process(image)

                            try:
                                pose_coor = results.pose_landmarks.landmark
                                # no need z here
                                body_kps = np.array([[landmark.x, landmark.y, landmark.visibility] for landmark in pose_coor]).flatten()
                                test_arr.append(body_kps)
                                labels.append(label)

                            except:
                                error_files.append(full_path)
                else:
                    image = Image.open(full_path)
                    image = np.array(image)
                    label = split[-2]
                    with mp_pose.Pose(min_detection_confidence = 0.5, min_tracking_confidence = 0.5) as pose:

                        results = pose.process(image)

                        try:
                            pose_coor = results.pose_landmarks.landmark
                            # no need z here
                            body_kps = np.array([[landmark.x, landmark.y, landmark.visibility] for landmark in pose_coor]).flatten()
                            test_arr.append(body_kps)
                            labels.append(label)

                        except:
                            error_files.append(full_path)
                        
else:
    for idx, row in label_df.iterrows():
        abs_path = os.path.join(ds_base_dir, row[0])
        arr = np.load(abs_path)
        if MODEL_TYPE == 'kps_stacked':
            arr = arr.flatten()
        test_arr.append(arr)
test_arr = np.array(test_arr)

0it [00:00, ?it/s]INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
INFO: Replacing 223 node(s) with delegate (TfLiteXNNPackDelegate) node, yielding 6 partitions.
INFO: Replacing 318 node(s) with delegate (TfLiteXNNPackDelegate) node, yielding 1 partitions.
INFO: Replacing 223 node(s) with delegate (TfLiteXNNPackDelegate) node, yielding 6 partitions.
INFO: Replacing 318 node(s) with delegate (TfLiteXNNPackDelegate) node, yielding 1 partitions.
INFO: Replacing 223 node(s) with delegate (TfLiteXNNPackDelegate) node, yielding 6 partitions.
INFO: Replacing 318 node(s) with delegate (TfLiteXNNPackDelegate) node, yielding 1 partitions.
INFO: Replacing 223 node(s) with delegate (TfLiteXNNPackDelegate) node, yielding 6 partitions.
INFO: Replacing 318 node(s) with delegate (TfLiteXNNPackDelegate) node, yielding 1 partitions.
INFO: Replacing 223 node(s) with delegate (TfLiteXNNPackDelegate) node, yielding 6 partitions.
INFO: Replacing 318 node(s) with delegate (TfLiteXNNPackDelegate) node,

In [13]:
test_arr.shape

(252, 99)

In [14]:
raw_pred = loaded_model.predict(test_arr)
pred = np.argmax(raw_pred, axis=1)
txt_pred = loaded_le.inverse_transform(pred)
np.unique(txt_pred)

2022-06-19 17:24:50.208567: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2022-06-19 17:24:50.225338: I tensorflow/core/platform/profile_utils/cpu_utils.cc:114] CPU Frequency: 3500050000 Hz
2022-06-19 17:24:50.286070: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublas.so.11
2022-06-19 17:24:50.606995: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublasLt.so.11


array(['squat-down', 'squat-up'], dtype='<U10')

In [15]:
from sklearn.metrics import accuracy_score

In [16]:
# take a peek at test acc
if MODEL_TYPE == 'kps_single':
    print(accuracy_score(labels, txt_pred))
else:
    print(accuracy_score(list(label_df['class']), txt_pred))

0.8690476190476191


In [99]:
final_df = label_df.copy()
final_df['predicted'] = txt_pred
final_df

,path,class,3-classes-view,5-classes-view,position,distance,predicted
0,keypoints_seq/len_8/half-burpee/half-burpee-in...,half-burpee-in,side,side,center,fit,half-burpee-in
1,keypoints_seq/len_8/half-burpee/half-burpee-in...,half-burpee-in,side,side,center,fit,half-burpee-in
2,keypoints_seq/len_8/half-burpee/half-burpee-in...,half-burpee-in,side,side,center,fit,half-burpee-in
3,keypoints_seq/len_8/half-burpee/half-burpee-in...,half-burpee-in,side,side,center,fit,half-burpee-in
4,keypoints_seq/len_8/half-burpee/half-burpee-in...,half-burpee-in,side,side,center,fit,half-burpee-in
...,...,...,...,...,...,...,...
2699,keypoints_seq/len_8/half-burpee/others/half-bu...,others,side,side,center,far,half-burpee-out
2700,keypoints_seq/len_8/half-burpee/others/half-bu...,others,side,side,center,far,half-burpee-out
2701,keypoints_seq/len_8/half-burpee/others/half-bu...,others,side,side,center,far,half-burpee-out
2702,keypoints_seq/len_8/half-burpee/others/half-bu...,others,side,side,center,far,half-burpee-out


In [100]:
final_df['class'].unique()

array(['half-burpee-in', 'half-burpee-out', 'idle', 'others'],
      dtype=object)

In [101]:
final_df['predicted'].unique()

array(['half-burpee-in', 'half-burpee-out'], dtype=object)

In [102]:
if BENCHMARK:
    output_file = os.path.join(output_dir, f'{MODEL_TYPE}_{EXERCISE}_BM.csv')
else:
    output_file = os.path.join(output_dir, f'{MODEL_TYPE}_{EXERCISE}.csv')
final_df.to_csv(output_file, index=False)

---

# Rep Counting

## Config

In [16]:
COUNTING_ORDER = {
    'pushup': {
        'cur_class': 'pushup-up',
        'prev_class': 'pushup-down'
    },
    'squat': {
        'cur_class': 'squat-up',
        'prev_class': 'squat-down'
    },
    'jumping-jack': {
        'cur_class': 'jumping-jack-down',
        'prev_class': 'jumping-jack-up'
    },
    'leg-raise': {
        'cur_class': 'leg-raise-down',
        'prev_class': 'leg-raise-up'
    },
    'half-burpee': {
        'cur_class': 'half-burpee-out',
        'prev_class': 'half-burpee-in'
    },
}

In [17]:
# model and counting
# EXERCISE = 'pushup'
# CUR_CLASS = 'up'
# PREV_CLASS = 'down'

MODEL_TYPE = 'kps_single'
# POST = 'hard_vote'
POST = 'None'
WINDOW_SIZE = 15 # window size for hard voting
BEST = False
ROUND_MAPPING = {
    'pushup': 8,
    'squat': 6,
    'jumping-jack': 4,
    'leg-raise': 3,
    'half-burpee': 3,
}
SEQ_LEN = 'single'
BENCHMARK = True

# input videos
DS_NAME = 'EzFit_dataset'
DS_BASE_DIR = f'/data1-6tb/jom/EzFit_models/evaluation_videos/{DS_NAME}'
VID_DIR = os.path.join(DS_BASE_DIR, 'videos')
OUT_DIR = os.path.join(DS_BASE_DIR, 'outputs')
GT_CSV = os.path.join(DS_BASE_DIR, 'labels/full_rep_gt.csv')

if MODEL_TYPE == 'ViViT':
    custom_objects = {"TubeletEmbedding": TubeletEmbedding, 
                      "PositionalEncoder": PositionalEncoder}
else:
    custom_objects = {}

## Run inference

In [18]:
mp_pose = mp.solutions.pose
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

In [19]:
df = pd.read_csv(GT_CSV)
df

,file_name,rep,exercise,3-classes-view,5-classes-view,position,distance
0,half-burpee_1_1.mp4,11,half-burpee,side,side,center,fit
1,half-burpee_1_2.mp4,9,half-burpee,side,side,center,fit
2,half-burpee_2_1.mp4,3,half-burpee,side,side,center,fit
3,half-burpee_2_2.mp4,4,half-burpee,side,right-tilted,right,fit
4,half-burpee_2_3.mp4,3,half-burpee,back,back,center,fit
...,...,...,...,...,...,...,...
63,squat_3_1.mp4,8,squat,side,side,center,fit
64,squat_3_2.mp4,6,squat,front,front,center,fit
65,squat_3_3.mp4,7,squat,side,right-tilted,center,far
66,squat_3_4.mp4,5,squat,side,side,center,far


In [20]:
cur_ex = None
prev_ex = None
all_obos = []
all_reps = []
for idx, row in df.iterrows():
    # 0 -> file_name, 1 -> rep, so on...
    file_name = row['file_name']
    rep_gt = row['rep']
    exercise = row['exercise']
    cur_ex = exercise
    rep_count = 0
    r = ROUND_MAPPING[exercise]
    if BENCHMARK:
        # r = 'BM'
        r = 'BM_BARE'
    # if exercise changes, reload model and le to corresponded exercise
    if cur_ex != prev_ex:
        if BEST:
            model_path = f'/data1-6tb/jom/EzFit_models/final_models/h5/{exercise}_{MODEL_TYPE}.h5'
            le_path = f'/data1-6tb/jom/EzFit_models/final_models/le/{exercise}_{MODEL_TYPE}_le.pickle'
        else:
            model_path = f'/data1-6tb/jom/EzFit_models/saved_models/len_{SEQ_LEN}/round_{r}/{exercise}/{exercise}_{MODEL_TYPE}.h5'
            if MODEL_TYPE == 'kps_single':
                le_path = f'/data1-6tb/jom/EzFit_models/saved_pickles/len_{SEQ_LEN}/round_{r}/{exercise}/{exercise}_{MODEL_TYPE}_lb.pickle'
            else:
                le_path = f'/data1-6tb/jom/EzFit_models/saved_pickles/len_{SEQ_LEN}/round_{r}/{exercise}/{exercise}_{MODEL_TYPE}_le.pickle'
        # load model and le
        loaded_model = load_model(model_path, custom_objects=custom_objects)
        with open(le_path, 'rb') as file:
            loaded_le = pickle.load(file)
    
    # start loading video and count
    if 'kps' in MODEL_TYPE: # kps-based models
        if r != 'BM_BARE':
            kps_seq = deque(maxlen=SEQ_LEN)
        current_stage = None
        previous_stage = None
        if POST == 'hard_vote':
            preds_window = deque(maxlen=WINDOW_SIZE)
        cap = cv2.VideoCapture(os.path.join(VID_DIR, exercise, file_name))
        with mp_pose.Pose(min_detection_confidence = 0.5, min_tracking_confidence = 0.5) as pose:
            while cap.isOpened():
                success, frame = cap.read()
                if not success:
                    break

                image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                results = pose.process(image)

                # append face&body coordinates for each frame to the csv file to create dataset to train the model
                try:
                    # extract pose
                    pose_coor = results.pose_landmarks.landmark
                    pose_row = np.array([[landmark.x, landmark.y, landmark.visibility] for landmark in pose_coor]).flatten()
                    # append new coor to the sequence array
                    
                    if MODEL_TYPE == 'kps_single':
                        y_hat = loaded_model.predict(tf.expand_dims(pose_row, axis=0))[0]
                        y_idx = np.argmax(y_hat)
                        if POST == 'hard_vote':
                            # do hard voting
                            preds_window.append(y_idx)
                            pred_count = Counter(preds_window)
                            # most_common return .items() format
                            # ex. [(0,5), (1,3)]
                            voted_pred = pred_count.most_common(1)[0][0]
                            class_name = loaded_le.classes_[voted_pred]
                        else:
                            class_name = loaded_le.classes_[y_idx]
                        
                    else: # kps_stacked and kps_seq
                        kps_seq.append(pose_row)
                        if len(kps_seq) == SEQ_LEN:
                            if MODEL_TYPE == 'kps_stacked':
                                seq_arr = np.array(kps_seq).flatten()
                            elif MODEL_TYPE == 'kps_seq':
                                seq_arr = np.array(kps_seq)
                            y_hat = loaded_model.predict(tf.expand_dims(seq_arr, axis=0))[0]
                            y_idx = np.argmax(y_hat)

                            if POST == 'hard_vote':
                                # do hard voting
                                preds_window.append(y_idx)
                                pred_count = Counter(preds_window)
                                # most_common return .items() format
                                # ex. [(0,5), (1,3)]
                                voted_pred = pred_count.most_common(1)[0][0]
                                class_name = loaded_le.classes_[voted_pred]
                            else:
                                class_name = loaded_le.classes_[y_idx]
                        else:
                            class_name = "None"

                    # count the rep logic
                    current_stage = class_name

                    if current_stage == COUNTING_ORDER[exercise]['cur_class'] and previous_stage == COUNTING_ORDER[exercise]['prev_class']:
                        rep_count += 1

                    previous_stage = current_stage  
                except Exception as e:
                    print(f'[INFO] error in {exercise}:', e)
            # after 1 vid end
            cap.release()
            
    else: # video_sequence, ViViT, image_single, Swin
        vid_seq = deque(maxlen=SEQ_LEN)
        current_stage = None
        previous_stage = None
        if POST == 'hard_vote':
            preds_window = deque(maxlen=WINDOW_SIZE)
        cap = cv2.VideoCapture(os.path.join(VID_DIR, exercise, file_name))
        while cap.isOpened():
            success, frame = cap.read()
            if not success:
                break

            image_input = cv2.resize(frame, (120, 120))

            if MODEL_TYPE in ['image_single', 'Swin']:
                y_hat = loaded_model.predict(tf.expand_dims(image_input, axis=0))[0]
                y_idx = np.argmax(y_hat)

                if POST == 'hard_vote':
                    # do hard voting
                    preds_window.append(y_idx)
                    pred_count = Counter(preds_window)
                    # most_common return .items() format
                    # ex. [(0,5), (1,3)]
                    voted_pred = pred_count.most_common(1)[0][0]
                    class_name = loaded_le.classes_[voted_pred]
                else:
                    class_name = loaded_le.classes_[y_idx]
                    
            else: # video_seqeunce, ViViT
                vid_seq.append(image_input)

                if len(vid_seq) == SEQ_LEN:
                    input_seq = np.array(vid_seq) / 255.0
                    y_hat = loaded_model.predict(tf.expand_dims(input_seq, axis=0))[0]
                    y_idx = np.argmax(y_hat)

                    if POST == 'hard_vote':
                        # do hard voting
                        preds_window.append(y_idx)
                        pred_count = Counter(preds_window)
                        # most_common return .items() format
                        # ex. [(0,5), (1,3)]
                        voted_pred = pred_count.most_common(1)[0][0]
                        class_name = loaded_le.classes_[voted_pred]
                    else:
                        class_name = loaded_le.classes_[y_idx]
                else:
                    class_name = "None"

            # count the rep logic
            current_stage = class_name

            if current_stage == COUNTING_ORDER[exercise]['cur_class'] and previous_stage == COUNTING_ORDER[exercise]['prev_class']:
                rep_count += 1

            previous_stage = current_stage
    
    print(f'{file_name}: {rep_count} from {rep_gt}')
    obo_score = OBO(rep_count, rep_gt)
    all_reps.append(rep_count)
    all_obos.append(obo_score)
    prev_ex = cur_ex
    
df['pred'] = all_reps
df['obo'] = all_obos
ds_name = DS_BASE_DIR.split(os.path.sep)[-1]
if BENCHMARK:
    df.to_csv(os.path.join(OUT_DIR, f'{MODEL_TYPE}_{POST}_{ds_name}_BM.csv'), index=False)
else:
    df.to_csv(os.path.join(OUT_DIR, f'{MODEL_TYPE}_{POST}_{ds_name}.csv'), index=False)

INFO: Replacing 223 node(s) with delegate (TfLiteXNNPackDelegate) node, yielding 6 partitions.
INFO: Replacing 318 node(s) with delegate (TfLiteXNNPackDelegate) node, yielding 1 partitions.


half-burpee_1_1.mp4: 11 from 11


INFO: Replacing 223 node(s) with delegate (TfLiteXNNPackDelegate) node, yielding 6 partitions.
INFO: Replacing 318 node(s) with delegate (TfLiteXNNPackDelegate) node, yielding 1 partitions.


half-burpee_1_2.mp4: 9 from 9


INFO: Replacing 223 node(s) with delegate (TfLiteXNNPackDelegate) node, yielding 6 partitions.
INFO: Replacing 318 node(s) with delegate (TfLiteXNNPackDelegate) node, yielding 1 partitions.


...

[INFO] error in squat: 'NoneType' object has no attribute 'landmark'
squat_3_3.mp4: 0 from 7


INFO: Replacing 223 node(s) with delegate (TfLiteXNNPackDelegate) node, yielding 6 partitions.
INFO: Replacing 318 node(s) with delegate (TfLiteXNNPackDelegate) node, yielding 1 partitions.


squat_3_4.mp4: 0 from 5


INFO: Replacing 223 node(s) with delegate (TfLiteXNNPackDelegate) node, yielding 6 partitions.
INFO: Replacing 318 node(s) with delegate (TfLiteXNNPackDelegate) node, yielding 1 partitions.


squat_3_5.mp4: 5 from 5


In [21]:
df

,file_name,rep,exercise,3-classes-view,5-classes-view,position,distance,pred,obo
0,half-burpee_1_1.mp4,11,half-burpee,side,side,center,fit,11,1
1,half-burpee_1_2.mp4,9,half-burpee,side,side,center,fit,9,1
2,half-burpee_2_1.mp4,3,half-burpee,side,side,center,fit,3,1
3,half-burpee_2_2.mp4,4,half-burpee,side,right-tilted,right,fit,4,1
4,half-burpee_2_3.mp4,3,half-burpee,back,back,center,fit,3,1
...,...,...,...,...,...,...,...,...,...
63,squat_3_1.mp4,8,squat,side,side,center,fit,8,1
64,squat_3_2.mp4,6,squat,front,front,center,fit,5,1
65,squat_3_3.mp4,7,squat,side,right-tilted,center,far,0,0
66,squat_3_4.mp4,5,squat,side,side,center,far,0,0


In [14]:
df.drop(['pred', 'obo'], axis=1,inplace=True)

In [15]:
df

,file_name,rep,3-classes-view,5-classes-view,sure,position,distance,gender,indoor/outdoor,special_case,exercise
0,v_JumpingJack_g01_c01.avi,3,front,front,yes,center,fit,female,indoor,NaN,jumping-jack
1,v_JumpingJack_g01_c02.avi,3,front,front,yes,center,fit,female,indoor,NaN,jumping-jack
2,v_JumpingJack_g01_c03.avi,3,front,front,yes,center,fit,female,indoor,NaN,jumping-jack
3,v_JumpingJack_g01_c04.avi,3,front,front,yes,center,fit,female,indoor,NaN,jumping-jack
4,v_JumpingJack_g01_c05.avi,3,front,front,yes,center,fit,female,indoor,NaN,jumping-jack
...,...,...,...,...,...,...,...,...,...,...,...
331,v_BodyWeightSquats_g25_c03.avi,1,side,side,yes,center,fit,male,indoor,NaN,squat
332,v_BodyWeightSquats_g25_c04.avi,1,side,side,yes,center,fit,male,indoor,NaN,squat
333,v_BodyWeightSquats_g25_c05.avi,1,side,side,yes,center,fit,male,indoor,NaN,squat
334,v_BodyWeightSquats_g25_c06.avi,1,side,side,yes,center,fit,male,indoor,NaN,squat


In [9]:
## NOTE
# all squat fail from front view (some pushup too)

# half-rep or worse exercise sample
# pushup-6 | pushup-13 | 

# jumping-jack fail from multi person and completely sideview